In [1]:
from sqlalchemy import types
from StringIO import StringIO
import sqlite3
import CleanData as cd
import numpy as np
import pandas as pd

# Clean Data

### British Museum

In [2]:
title = 'AugustusCoins_44BC-14AD'

In [3]:
# Read in data and convert to appropriate types
bm_df = pd.read_csv('../Data/BM_'+title+'.csv',
                 converters={"Authority": cd.stringToList(), 'Associated names': cd.stringToList(), 
                             'Subjects': cd.stringToList(), 'Inscriptions': cd.stringToListofDicts('|', ';', ':')})
bm_df = pd.concat([bm_df, cd.cleanInscriptions(bm_df)], axis=1)
bm_df = bm_df.drop('Unnamed: 0', 1)
bm_df = bm_df.replace(np.nan, '', regex=True)
bm_df = bm_df.rename(columns = {'Culture/period':'Culture'})

In [4]:
# Clean data to only retain relevant information
mask = ((bm_df['Object type'] == 'coin ') & (bm_df['Date'].str.find('stC') == -1))
filtered = bm_df[mask]

lists = ['Authority', 'Subjects', 'Associated names', 'Obverse legend', 'Reverse legend']
strings = ['Museum number', 'Denomination', 'Description', 'State', 'Culture', 'Materials', 
            'Curator\'s comments', 'Bibliography', 'Object type', "Production place"]
floats = ['Weight (g)']
dates = ['Date']
redundant_notes = ['Production place', 'Denomination']
do_nothing = ['url', 'Inscriptions']
duplicate_cols = ['Authority', 'Date', 'Production place', 'Description', 'Subjects', "Curator's comments",
                  'Obverse legend', 'Reverse legend']

cleaned_bm = cd.cleanDF(filtered, lists, strings, floats, dates, redundant_notes, do_nothing, duplicate_cols)
cleaned_bm = cleaned_bm.drop('Inscriptions', 1)
cleaned_bm['Source'] = 'British Museum'
#cleaned_bm.head()

In [5]:
# Convert everything to strings and integers to put into SQL db
lists = ['Authority', 'Subjects', 'Associated names', 'Obverse legend', 'Reverse legend', 'Date']
for col in lists:
    cleaned_bm[col] = cleaned_bm[col].apply(cd.listToString())

columns = [u'Associated names', u'Authority', u'Bibliography', u'Culture',
       u'Curator\'s comments', u'Date', u'Denomination', u'Description',
       u'Materials', u'Museum number', u'Object type', u'Obverse legend',
       u'Production place', u'Reverse legend', u'State', u'Subjects',
       u'url', u'Source']
for col in columns:
    cleaned_bm[col] = cleaned_bm[col].apply(lambda x: x.decode('utf-8'))
    
cleaned_bm.columns = ['associatedNames', 'authority', 'bibliography', 'culture',
                       'curatorComment', 'date', 'denomination', 'description',
                       'materials', 'museumNumber', 'objectType', 'obverseLegend',
                       'mint', 'reverseLegend', 'state', 'subjects',
                       'weight', 'url', 'source']

In [6]:
cleaned_bm.tail()

,associatedNames,authority,bibliography,culture,curatorComment,date,denomination,description,materials,museumNumber,objectType,obverseLegend,mint,reverseLegend,state,subjects,weight,url,source
1527,"Mark Antony, Octavia, Augustus (Octavian)","M Oppius Capito, Mark Antony","RPC1 1463 (type) RR2 154, p.518",Roman Republican,,"-38, -37",tressis,Copper alloy coin.(obverse) Busts of M. Antoni...,copper alloy,"1860,0328.250",coin,[M ANT IMP TERT COS DESIG ITER ET TER ...,Achaea,", [M OPPIVS CAPITO PRO PR PRAEF CLASS F...",Roman Republic,"politician/statesman, emperor/empress, boat/ship",21.51,http://www.britishmuseum.org/research/collecti...,British Museum
1528,"Mark Antony, Augustus (Octavian)","Mark Antony, M Barbatius Pollio","Ghey, Leins & Crawford 2010 517.2.12 RRC 517/2...",Roman Republican,"Mint uncertain, moving with Mark Antony.Not th...",-41,denarius,Silver coin.(obverse) Head of M. Antonius righ...,silver,"1932,0706.24.3",coin,M ANT IMP AVG III VIR R P C M BARBAT...,,CAESAR IMP PONT III VIR R P C,Roman Republic,"politician/statesman, emperor/empress",3.39,http://www.britishmuseum.org/research/collecti...,British Museum
1529,"Marcus Aemilius Lepidus, Augustus (Octavian)",Marcus Aemilius Lepidus,"Ghey, Leins & Crawford 2010 495.2.6 RRC 495/2a...",Roman Republican,Die appears to read IMA (ligatured) instead of...,-42,denarius,Silver coin.(obverse) Head of M. Lepidus right...,silver,"2011,4027.3",coin,LEPIDVS PONT MAX III V R P C,Italy,CAESAR IMA III VIR R P C,Roman Republic,"politician/statesman, emperor/empress",3.56,http://www.britishmuseum.org/research/collecti...,British Museum
1530,"Augustus (Octavian), Eros/Cupid, Aphrodite/Venus",P Clodius,"RR1 4277, p.583 Ghey, Leins & Crawford 2010 49...",Roman Republican,,-42,aureus,Gold coin; pierced for suspension.(obverse) He...,gold,"1852,0903.11",coin,C CAESAR III VIR R P C,Rome,P CLODIVS M F IIII VIR A P F,Roman Republic,"cherub/cupid, politician/statesman, classical ...",8.09,http://www.britishmuseum.org/research/collecti...,British Museum
1531,"Augustus (Octavian), Tyche/Fortuna",Ti Sempronius Gracchus,"RR1 4313, p.593 Ghey, Leins & Crawford 2010 52...",Roman Republican,,-40,aureus,"Gold coin.(obverse) Head of Octavian right, be...",gold,"1844,0425.473",coin,"DIVI, IVLI F",Rome,"IIII VIR Q D, TI SEMPRON GRACCVS",Roman Republic,"politician/statesman, allegory/personification...",7.96,http://www.britishmuseum.org/research/collecti...,British Museum


### American Numismatic Society

In [7]:
ans_df = pd.read_csv('../Data/ANS_'+title+'.csv',
                    converters={"Authority": cd.stringToList('|'), 'Year': cd.stringToList('|'),
                               'Issuer': cd.stringToList('|'), 'Portrait': cd.stringToList('|'),
                               'Reference': cd.stringToList('|')})
ans_df = ans_df.replace(np.nan, '', regex=True)
ans_df['Description'] = ans_df.apply(lambda row: cd.makeDescription(row['Material'], row['Manufacture'], row['Obverse Type'], row['Reverse Type']), axis=1)
#ans_df.tail()

In [8]:
lists = ['Year', 'Authority', 'Issuer', 'Portrait', 'Reference']
strings = ['Deity', 'Denomination', 'Mint', 'Description', 'Obverse Legend', 'Reverse Legend']
floats = ['Weight']
dates = []
redundant_notes = []
do_nothing = ['URI']
duplicate_cols = ['Issuer', 'Year', 'Mint', 'Deity', 'Description', 'Obverse Legend', 'Reverse Legend']

cleaned_ans = cd.cleanDF(ans_df, lists, strings, floats, dates, redundant_notes, do_nothing, 
                              duplicate_cols, production_place='Mint')
cleaned_ans['Source'] = 'British Museum'
cleaned_ans.tail()
cleaned_ans.columns

Index([u'Authority', u'Deity', u'Denomination', u'Description', u'Issuer',
       u'Mint', u'Obverse Legend', u'Portrait', u'Reference',
       u'Reverse Legend', u'URI', u'Weight', u'Year', u'Source'],
      dtype='object')

In [9]:
# Convert everything to strings and integers to put into SQL db
lists = ['Year', 'Authority', 'Issuer', 'Portrait', 'Reference']
for col in lists:
    cleaned_ans[col] = cleaned_ans[col].apply(cd.listToString())

columns = [u'Authority', u'Deity', u'Denomination', u'Description', u'Issuer',
       u'Mint', u'Obverse Legend', u'Portrait', u'Reference',
       u'Reverse Legend', u'URI', u'Year', u'Source']
for col in columns:
    cleaned_ans[col] = cleaned_ans[col].apply(lambda x: x.decode('utf-8'))

cleaned_ans.head()

,Authority,Deity,Denomination,Description,Issuer,Mint,Obverse Legend,Portrait,Reference,Reverse Legend,URI,Weight,Year,Source
0,Augustus,,As,Struck Bronze. (obverse) Augustus head right. ...,,Ephesus,CAESAR,Augustus,RIC I (second edition) Augustus 486,AVGVSTVS,http://numismatics.org/search/id/1944.100.39200,11.10,-25,British Museum
1,Augustus,,As,Struck Bronze.,,Ephesus,CAESAR,Augustus,"BMC.731, RIC I (second edition) Augustus 486",AVGVSTVS,http://numismatics.org/search/id/1952.142.117,10.92,-25,British Museum
2,Augustus,,Aureus,Struck Gold.,,Colonia Patricia,,Augustus,"BMC.345v, RIC I (second edition) Augustus 50A",,http://numismatics.org/search/id/1957.172.1489,3.66,"-20, -19",British Museum
3,Augustus,,Aureus,"Struck Gold. (obverse) Head of Augustus, laure...",,Lugdunum,AVGVSTVS DIVI F,Augustus,"BMC.492, RIC I (second edition) Augustus 200",IMP XIIII,http://numismatics.org/search/id/0000.999.16779,7.88,-9,British Museum
4,Augustus,,Denarius,Bronze. (obverse) Bare head r.. (reverse) Fema...,Augustus,,AVGVSTVS DIVI F,Augustus,,IMP X ACT,http://numismatics.org/search/id/1944.27.9,0.00,"-26, 14",British Museum


# Store Data

In [10]:
cnx = sqlite3.connect('../Data/'+title+'.sqlite')
cur = cnx.cursor()

In [11]:
cur.execute('''CREATE TABLE britishMuseum(associatedNames TEXT, authority TEXT, bibliography TEXT, culture TEXT,
                    curatorComment TEXT, dates TEXT, denomination TEXT, description TEXT,
                    materials TEXT, museumNumber TEXT, objectType TEXT, obverseLegend TEXT,
                    mint TEXT, reverseLegend TEXT, state TEXT, subjects TEXT, weight INT,
                    url TEXT, source TEXT)''')

In [12]:
cur.executemany("""INSERT INTO britishMuseum (associatedNames, authority, bibliography, culture,
                        curatorComment, dates, denomination, description,
                        materials, museumNumber, objectType, obverseLegend,
                        mint, reverseLegend, state, subjects, weight,
                        url, source) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""", 
                list(cleaned_bm.to_records(index=False)))

In [13]:
cur.execute('''CREATE TABLE americanNumismaticSociety(authority TEXT, deity TEXT, denomination TEXT, description TEXT, 
                       issuer TEXT, mint TEXT, obverseLegend TEXT, portrait TEXT, reference TEXT,
                       reverseLegend TEXT, url TEXT, weight INT, dates TEXT, source TEXT)''')

In [14]:
cur.executemany("""INSERT INTO americanNumismaticSociety (authority, deity, denomination, description, 
                       issuer, mint, obverseLegend, portrait, reference,
                       reverseLegend, url, weight, dates, source) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?)""", 
                list(cleaned_ans.to_records(index=False)))

In [15]:
cnx.commit()